In [8]:
# !pip3 install langchain
# !pip3 install openai
# !pip3 install chromadb
# !pip3 install pypdf

387.26s - pydevd: Sending message related to process being replaced timed-out after 5 seconds
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 515.4/515.4 kB 6.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 11.5 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.9/332.9 kB 12.5 MB/s eta 0:00:00
  Using cached dataclasses_json-0.5.7-py3-none-any.whl (25 kB)
  Using cached openapi_schema_pydantic-1.2.4-py3-none-any.whl (90 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 13.4 MB/s eta 0:00:00a 0:00:01
  Using cached requests-2.28.2-py3-none-any.whl (62 kB)
  Using cached tenacity-8.2.2-py3-none-any.whl (24 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.7/121.7 kB 12.0 MB/s eta 0:00:00
  Using cached async_timeout-4.0.2-py3-none-any.whl (5.8 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 kB 6.0 MB/s eta 0:00:00
  Using cached aiosignal-1.3.1-py3-none-any.whl (7.6 kB)
  Using cached marshmallow-3.19.0-p

In [9]:
# import os
# os.environ["OPENAI_API_KEY"] = "..."

In [2]:
from langchain.document_loaders import PyPDFLoader 
from langchain.embeddings import OpenAIEmbeddings 
from langchain.vectorstores import Chroma 
from langchain.chains import ConversationalRetrievalChain 
from langchain.llms import OpenAI 

In [45]:
pdf_path = "Swisslog_Testfile.pdf"
loader = PyPDFLoader(pdf_path)
pages = loader.load_and_split()
print(pages[1].page_content)

Nexus 7.4.x Software changes the Communication of our Devices and SCC from 
the SCC talking to the End Devices to the End Devices talking to the SCC.  To 
make this change, we updated the End Devices to work off of TCP.  There are a 
few ways that this can be done.  
 
1. When upgrading on the same SCC, no extra steps should be necessary.  
 
2. When upgrading and moving to a new SCC (called a Migration)  you should 
install the same version of software  on the Old SCC, pull a Database backup  
and restore the Database backup  on the New SCC  (the IP addresses are 
contained inside.)  
 
3. For a clean install, you should drop the Equip. CFG  file into place and import 
the IP addresses.   You may need to run the sl -Msg2Fld tool (see below)  
 
4. For moving to a Cold Backup  SCC , you either change the IP address of the 
Backup to match the SCC or run the sl -Msg2Fld tool  (see below)  
 
5. For a new device install, you set the server address at each device when 
installed or  confi

In [43]:
embeddings = OpenAIEmbeddings()
vectordb = Chroma.from_documents(pages, embeddings=embeddings, persist_directory=".")

Using embedded DuckDB with persistence: data will be stored in: .
No embedding_function provided, using default embedding function: SentenceTransformerEmbeddingFunction


In [50]:
qa = ConversationalRetrievalChain.from_llm(OpenAI(temperature=0.9, model_name="gpt-3.5-turbo"),
                                    vectordb.as_retriever(search_kwargs={"k": 1}), return_source_documents=True)

In [51]:
#turn one 
chat_history = []
query = "What should I do for a clean install?"
result = qa({"question": query, "chat_history": chat_history})
result['answer']

'For a clean install, you should drop the Equip. CFG file into place and import the IP addresses. You may need to run the sl -Msg2Fld tool.'

In [56]:
#turn two
chat_history = [(query, result["answer"])]
query = "Did they mention the tool ?"
result = qa({"question": query, "chat_history": chat_history})
result['answer']

'The tool that should be run for a clean install is the sl -Msg2Fld tool.'

In [54]:
# continous exec
query = " "
chat_history = []
while (text != ""):
    query = input()
    result = qa({"question": query, "chat_history": chat_history})
    chat_history.append((query, result["answer"]))
    print(result['answer'])

 What's the name?


The tool that should be run for a clean install is the sl -Msg2Fld tool.


 Where should I go for it 


The tool that should be run for a clean install is the sl -Msg2Fld tool.


The tool that should be run for a clean install is the sl -Msg2Fld tool.
